In [9]:
import requests
from lxml import etree
import pandas as pd
import os
import json

In [10]:
path =  'new_weibo_sample_525_4.csv'
data = pd.read_csv(path)
data.head()

,发布者ID,发布时间,发布来源,发布内容,收藏数,转发数,评论数,点赞数,verified_reason,description
0,新闻一加一,05月22日00:07,来自微博视频号,【当我们谈论反食品浪费的时候，到底在谈论什么？】主持人王宁：从开设“食物银行”到售卖“临期...,0,58,16,264,媒体人刘楠，著《新闻撞武侠》《有种基因叫理想》,《寻找白岩松》作者，央视十年记者，《新闻1+1》起名者、探村博士联盟发起人，公号：南瓜博士与...
1,成都新闻广播,05月22日09:56,来自iPhone客户端,#成都#【逛科技市集，感受科技魅力！】5月21日晚，“2021四川省科技活动周暨蓉城科学之...,0,0,0,1,成都电台资讯998官方微博,成都人民广播电台新闻频率官方微博。网络收听：www.cdbs.com.cn 本地收听FM9...
2,浦东发布,05月22日10:23,来自微博视频号,【乍看青铜再看王者系列：#男子写人名倒过来是肖像#，寥寥数笔翻转后超惊艳】据漩涡视频，5月...,0,3,0,15,上海市浦东新区人民政府新闻办公室官方微博,浦东的衣食住行、吃喝玩乐、大政小事，点点滴滴。。。这里没有围墙，私信全开放。
3,新闻酸菜馆,05月22日10:54,来自iPhone客户端,//分享网易新闻:《湖南长沙：袁隆平同志逝世》O湖南长沙：袁隆平同志逝世@网易新闻客户端#...,0,0,3,0,网络播客《新闻酸菜馆》官方微博,听新闻就上酸菜馆！品牌合作请私信联系，或邮件d@wasai.org
4,微博财经7X24小时,05月22日10:56,来自微博股票,【湖南长沙：袁隆平同志逝世】“杂交水稻之父”、中国工程院院士、“共和国勋章”获得者袁隆平因...,0,0,0,1,NaN,NaN


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10603 entries, 0 to 10602
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   发布者ID            10603 non-null  object
 1   发布时间             10603 non-null  object
 2   发布来源             10024 non-null  object
 3   发布内容             10603 non-null  object
 4   收藏数              10603 non-null  int64 
 5   转发数              10603 non-null  int64 
 6   评论数              10603 non-null  int64 
 7   点赞数              10603 non-null  int64 
 8   verified_reason  10013 non-null  object
 9   description      8312 non-null   object
dtypes: int64(4), object(6)
memory usage: 828.5+ KB


In [12]:
data['发布者ID'].value_counts()

吉林资讯广播       60
江川社情         34
吉林交通广播       31
徽州反邪教        29
抚州临川发布       29
             ..
河南青天河         1
平凉市广播电视台      1
精彩洛阳          1
临沂市消防救援支队     1
新津妇联          1
Name: 发布者ID, Length: 3901, dtype: int64

In [14]:
null_name = []
notnull_name = []
error_name = []
for name,gb_data in data.groupby('发布者ID'):
    # print(name,len(gb_data))
    sum = gb_data['verified_reason'].isnull().sum()
    # print(name,len(gb_data) == sum)
    # 缺失数据就是发布者ID数据
    if len(gb_data) == sum:
        null_name.append(name)
    else:
        if sum != 0:
            print(name,'有问题')
            error_name.append(name)
        else:
            null_name.append(name)

In [15]:
dict_data = {}
for name,gb_data in data.groupby('发布者ID'):
    if name in error_name:
        x = gb_data['verified_reason'][gb_data['verified_reason'].isnull() == False]
        y = gb_data['description'][gb_data['description'].isnull() == False]
        dict_data[name] = {}
        dict_data[name]['verified_reason'] = x.values[0]
        dict_data[name]['description'] = y.values[0] if len(y.values) != 0 else "无记录"

In [16]:
dict_data

{}

In [79]:
information = dict_data
for i in range(len(data)):
    name = data.loc[i,"发布者ID"]
    if name in information.keys():
        data.loc[i,'verified_reason'] = information[name]['verified_reason']
        data.loc[i,'description'] = information[name]['description']
        with open('names.txt','a+') as f:
            f.write('{}\n'.format(name))
    else:
        pass
data.to_csv('new_weibo_sample_525_3.csv',encoding='utf_8_sig',index=False)

In [17]:
data.verified_reason

0        媒体人刘楠，著《新闻撞武侠》《有种基因叫理想》
1                  成都电台资讯998官方微博
2           上海市浦东新区人民政府新闻办公室官方微博
3                网络播客《新闻酸菜馆》官方微博
4                            NaN
                  ...           
10598               共青团商洛委员会官方微博
10599                    新京报官方微博
10600               共青团商洛委员会官方微博
10601                  邯郸新闻网官方微博
10602         甘肃省张掖市民乐县人民检察院官方微博
Name: verified_reason, Length: 10603, dtype: object

In [87]:
data

,发布者ID,发布时间,发布来源,发布内容,收藏数,转发数,评论数,点赞数,verified_reason,description
0,新闻一加一,05月22日00:07,来自微博视频号,【当我们谈论反食品浪费的时候，到底在谈论什么？】主持人王宁：从开设“食物银行”到售卖“临期...,0,58,16,264,媒体人刘楠，著《新闻撞武侠》《有种基因叫理想》,《寻找白岩松》作者，央视十年记者，《新闻1+1》起名者、探村博士联盟发起人，公号：南瓜博士与...
1,成都新闻广播,05月22日09:56,来自iPhone客户端,#成都#【逛科技市集，感受科技魅力！】5月21日晚，“2021四川省科技活动周暨蓉城科学之...,0,0,0,1,成都电台资讯998官方微博,成都人民广播电台新闻频率官方微博。网络收听：www.cdbs.com.cn 本地收听FM9...
2,浦东发布,05月22日10:23,来自微博视频号,【乍看青铜再看王者系列：#男子写人名倒过来是肖像#，寥寥数笔翻转后超惊艳】据漩涡视频，5月...,0,3,0,15,上海市浦东新区人民政府新闻办公室官方微博,浦东的衣食住行、吃喝玩乐、大政小事，点点滴滴。。。这里没有围墙，私信全开放。
3,新闻酸菜馆,05月22日10:54,来自iPhone客户端,//分享网易新闻:《湖南长沙：袁隆平同志逝世》O湖南长沙：袁隆平同志逝世@网易新闻客户端#...,0,0,3,0,网络播客《新闻酸菜馆》官方微博,听新闻就上酸菜馆！品牌合作请私信联系，或邮件d@wasai.org
4,微博财经7X24小时,05月22日10:56,来自微博股票,【湖南长沙：袁隆平同志逝世】“杂交水稻之父”、中国工程院院士、“共和国勋章”获得者袁隆平因...,0,0,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
10598,商洛团市委,05月22日22:59,来自微博视频号,【袁隆平老友：“他走的时候很安详”，家属：几天前通过电话，他心情不错】22日下午，极目新闻...,0,0,0,6,共青团商洛委员会官方微博,共青团商洛委员会官方微博
10599,新京报,05月22日22:59,来自微博视频号,【泪目怀念！#视频回顾袁隆平的90后追梦人生#】“我有两个梦，一个是禾下乘凉梦；第二个是杂...,0,91,31,453,新京报官方微博,一份以责任为灵魂的报纸；一份致力于记录时代步伐的报纸；一份进步、美好的报纸。新京报，品质源于责任。
10600,商洛团市委,05月22日22:59,来自微博视频号,【#殡仪馆外市民排队为袁老献花#】今夜，长沙明阳山殡仪馆外，市民自发排队，为袁老献花，寄托...,0,0,0,2,共青团商洛委员会官方微博,共青团商洛委员会官方微博
10601,邯郸新闻网,05月22日22:59,来自iPhone客户端,【写满了爱！#袁隆平给妈妈写的歌#】袁隆平曾说，大家都说他用一粒种子改变了世界，其实，这粒...,0,0,0,0,邯郸新闻网官方微博,邯郸新闻网由邯郸日报社主管主办。


In [18]:
null_name = []
notnull_name = []
error_name = []
for name,gb_data in data.groupby('发布者ID'):
    # print(name,len(gb_data))
    sum = gb_data['verified_reason'].isnull().sum()
    # print(name,len(gb_data) == sum)
    # 缺失数据就是发布者ID数据
    if sum == 0:
        notnull_name.append(name)
    else:
        null_name.append(name)

In [19]:
len(null_name)

241

In [20]:
with open('error_name.txt','w') as f:
    for name in null_name:
        f.write(name+'\n')

In [106]:
def get_url():
    response = requests.get('https://s.weibo.com/user',headers=headers,params={'q':name,"Refer":"weibo_user"})
    tree = etree.HTML(response.text)
    names_url = tree.xpath('//div[@id="pl_user_feedList"]//div[@class="info"]//a//@href') # 找到名字，并且选取第一个
    return names_url

In [107]:
def get_information(name):
    if name in information.keys():
        return
    else:
        information[name] = {"verified_reason":"","description":""}
    names_url = get_url()
    try:
        response = requests.get('https://weibo.com/ajax/profile/info',headers=headers,params={'custom':names_url[0].split('/')[-1]})
    except Exception as e:
        # print(e)
        error_names.append(name)
        print(name + "未找到")
        print("已经成功爬取了{}条数据".format(len(information)))

        information[name]['verified_reason'] = "未找到" if information[name]['verified_reason'] == "" else information[name]['verified_reason']
        information[name]['description'] = "未找到" if information[name]['description'] == "" else information[name]['description']
        # break
        return
    
    info = json.loads(response.text)["data"]["user"]
    
    verified_reason = info["verified_reason"] if "verified_reason" in info else "无认证"
    description = info["description"] if "description" in info else "无介绍"
    
    information[name]['verified_reason'] = verified_reason
    information[name]['description'] = description
    if len(information) % 100 == 0:
        print("已经成功爬取了{}条数据".format(len(information)))

In [108]:
information = {}
error_names = []
weibo_name = data.发布者ID
from multiprocessing.dummy import Pool
if __name__ == '__main__':
    pool = Pool(8)
    pool.map(get_information,weibo_name.to_list())   

NameError: name 'headers' is not defined

In [ ]:
for name in error_names:
    if name in information.keys():
        continue
    else:
        information[name] = {"verified_reason":"","description":""}
    names_url = get_url()
    try:
        response = requests.get('https://weibo.com/ajax/profile/info',headers=headers,params={'custom':names_url[0].split('/')[-1]})
    except Exception as e:
        # print(e)
        error_names.append(name)
        print(name + "未找到")
        print("已经成功爬取了{}条数据".format(len(information)))

        information[name]['verified_reason'] = "未找到" if information[name]['verified_reason'] == "" else information[name]['verified_reason']
        information[name]['description'] = "未找到" if information[name]['description'] == "" else information[name]['description']
        # break
        continue
    
    info = json.loads(response.text)["data"]["user"]
    
    verified_reason = info["verified_reason"] if "verified_reason" in info else "无认证"
    description = info["description"] if "description" in info else "无介绍"
    
    information[name]['verified_reason'] = verified_reason
    information[name]['description'] = description
    
    if len(information) % 100 == 0:
        print("已经成功爬取了{}条数据".format(len(information)))

In [9]:
len(information)

81

In [10]:
verified_reasons = ["" for i in range(len(data))]
descriptions = ["" for i in range(len(data))]
data['verified_reason'] = verified_reasons
data['description'] = descriptions

In [13]:
for i in range(len(data)):
    name = data.loc[i,"发布者ID"]
    if name in information.keys():
        data.loc[i,'verified_reason'] = information[name]['verified_reason']
        data.loc[i,'description'] = information[name]['description']

In [50]:
import pandas as pd
import numpy as np

In [51]:
path =  'weibo_05.22-06.22_sort.csv'
data = pd.read_csv(path)
data

,发布者ID,发布时间,发布来源,发布内容,收藏数,转发数,评论数,点赞数
0,新闻一加一,05月22日00:07,来自微博视频号,【当我们谈论反食品浪费的时候，到底在谈论什么？】主持人王宁：从开设“食物银行”到售卖“临期...,0,58,16,264
1,成都新闻广播,05月22日09:56,来自iPhone客户端,#成都#【逛科技市集，感受科技魅力！】5月21日晚，“2021四川省科技活动周暨蓉城科学之...,0,0,0,1
2,浦东发布,05月22日10:23,来自微博视频号,【乍看青铜再看王者系列：#男子写人名倒过来是肖像#，寥寥数笔翻转后超惊艳】据漩涡视频，5月...,0,3,0,15
3,新闻酸菜馆,05月22日10:54,来自iPhone客户端,//分享网易新闻:《湖南长沙：袁隆平同志逝世》O湖南长沙：袁隆平同志逝世@网易新闻客户端#...,0,0,3,0
4,微博财经7X24小时,05月22日10:56,来自微博股票,【湖南长沙：袁隆平同志逝世】“杂交水稻之父”、中国工程院院士、“共和国勋章”获得者袁隆平因...,0,0,0,1
...,...,...,...,...,...,...,...,...
45446,吕梁中院,06月22日22:15,来自微博weibo.com,【#他们当年这样报志愿#】1937年，许渊冲考入西南联大外语系；1949年，袁隆平考入西南...,0,0,0,0
45447,丽江日报,06月22日22:16,来自HUAWEIP30,【#他们当年这样报志愿#】1937年，许渊冲考入西南联大外语系；1949年，袁隆平考入西南...,0,0,1,5
45448,上海反邪教,06月22日22:34,来自iPhone客户端,【#他们当年这样报志愿#】1937年，许渊冲考入西南联大外语系；1949年，袁隆平考入西南...,0,3,0,8
45449,平安银川,06月22日22:36,来自OnePlus8,【#他们当年这样报志愿#】1937年，许渊冲考入西南联大外语系；1949年，袁隆平考入西南...,0,0,0,0


In [52]:
df1 = data[data['发布时间'] <= "05月29日"]
df1

,发布者ID,发布时间,发布来源,发布内容,收藏数,转发数,评论数,点赞数
0,新闻一加一,05月22日00:07,来自微博视频号,【当我们谈论反食品浪费的时候，到底在谈论什么？】主持人王宁：从开设“食物银行”到售卖“临期...,0,58,16,264
1,成都新闻广播,05月22日09:56,来自iPhone客户端,#成都#【逛科技市集，感受科技魅力！】5月21日晚，“2021四川省科技活动周暨蓉城科学之...,0,0,0,1
2,浦东发布,05月22日10:23,来自微博视频号,【乍看青铜再看王者系列：#男子写人名倒过来是肖像#，寥寥数笔翻转后超惊艳】据漩涡视频，5月...,0,3,0,15
3,新闻酸菜馆,05月22日10:54,来自iPhone客户端,//分享网易新闻:《湖南长沙：袁隆平同志逝世》O湖南长沙：袁隆平同志逝世@网易新闻客户端#...,0,0,3,0
4,微博财经7X24小时,05月22日10:56,来自微博股票,【湖南长沙：袁隆平同志逝世】“杂交水稻之父”、中国工程院院士、“共和国勋章”获得者袁隆平因...,0,0,0,1
...,...,...,...,...,...,...,...,...
40729,国是直通车,05月28日22:45,NaN,#洞见计划#【#中国共产党的十万个为什么#|为什么袁隆平申请研究经费常常是“秒到账”？】众...,0,3,0,9
40730,各界导报社官微,05月28日22:47,来自vivo全新水滴屏手机,【天津#网民侵害袁隆平名誉被立案调查#】2021年5月22日，“杂交水稻之父”、中国工程院...,0,0,0,0
40731,红网,05月28日22:56,来自微博weibo.com,【一句话一辈子！一组海报忆袁老】音容宛在，功勋永存；先生千古，国士无双！听8位袁隆平院士生...,0,10,2,83
40732,益路繁星,05月28日22:57,来自微博视频号,【点赞！#农民小哥用玉米画出袁隆平#】5月27日，河南商丘，农民小哥姜亮用玉米画出袁隆平爷...,0,0,0,1


In [53]:
df2 = data[data['发布时间'] > "05月29日"]
df2

,发布者ID,发布时间,发布来源,发布内容,收藏数,转发数,评论数,点赞数
40734,天津政法,05月29日03:00,来自微博weibo.com,【#中国共产党的十万个为什么#为什么袁隆平申请研究经费常常是“秒到账”？】#中国共产党一百...,0,2,1,1
40735,明水检察,05月29日04:03,来自Redmi9A,:【#天津网民侵害袁隆平院士名誉被检察公益诉讼立案调查#】,0,8,17,17
40736,慈溪电视台关注栏目,05月29日06:27,来自微博weibo.com,#缅怀#【为什么袁隆平伟大?粮食梦不能丢!】总书记高度肯定袁隆平同志对我国粮食安全农业科技...,0,2,0,2
40737,吉林资讯广播,05月29日07:09,来自微博视频号,【广东#小伙用镰刀雕刻袁隆平巨幅墙绘#：镰刀是收割水稻的工具】5月28日，广东河源。一面旧...,0,0,0,2
40738,济宁高新检察,05月29日07:20,来自iPhone客户端,【天津检察机关依法对某网民侵害袁隆平院士名誉的行为开展公益诉讼立案调查】5月22日，“杂交...,0,12,5,10
...,...,...,...,...,...,...,...,...
45446,吕梁中院,06月22日22:15,来自微博weibo.com,【#他们当年这样报志愿#】1937年，许渊冲考入西南联大外语系；1949年，袁隆平考入西南...,0,0,0,0
45447,丽江日报,06月22日22:16,来自HUAWEIP30,【#他们当年这样报志愿#】1937年，许渊冲考入西南联大外语系；1949年，袁隆平考入西南...,0,0,1,5
45448,上海反邪教,06月22日22:34,来自iPhone客户端,【#他们当年这样报志愿#】1937年，许渊冲考入西南联大外语系；1949年，袁隆平考入西南...,0,3,0,8
45449,平安银川,06月22日22:36,来自OnePlus8,【#他们当年这样报志愿#】1937年，许渊冲考入西南联大外语系；1949年，袁隆平考入西南...,0,0,0,0


In [54]:
total = 526
result = pd.concat([df1.sample(int(total*0.8)),df2.sample(int(total*0.2))],ignore_index=True)
result

,发布者ID,发布时间,发布来源,发布内容,收藏数,转发数,评论数,点赞数
0,内蒙古团委,05月23日13:53,来自想见你的Android,【#袁隆平遗体送别仪式明天举行#】5月22日，共和国痛失袁隆平院士。记者从袁隆平同志治丧办...,0,1,1,3
1,正安县检察院,05月25日14:40,来自iPhone客户端,【#女老师哭着读学生的信追思袁老#：当风吹过稻田，我就知道您回来了】#市民赴三亚试验田献花...,0,0,1,0
2,沈阳网警巡查执法,05月24日14:32,来自微博视频号,【#聋哑男孩跑步作画致敬袁隆平#】来自内蒙古自治区赤峰市一名聋哑男孩吴广明，在得知袁隆平院...,0,0,6,5
3,sxrtv-新闻中心,05月22日14:38,来自微博视频号,【痛悼！#送别国之脊梁袁隆平#】今天，#共和国痛失袁隆平院士#。“我最大的愿望是饭碗要牢牢...,0,0,0,0
4,烟台高速交警,05月23日07:16,来自nova7你在焦点在,#送别国之脊梁袁隆平#国士无双……​,0,0,0,1
...,...,...,...,...,...,...,...,...
520,如皋教育发布,05月30日08:16,来自小米6拍人更美,【他们，才是#中国人最该追的星#！致敬！】“憋着一肚子气”搞导弹研发的钱学森、“要求自己鞠...,0,0,0,0
521,江岸检察,06月22日09:46,来自微博视频号,【#袁隆平儿子接棒继续袁梦计划#】距离袁隆平院士逝世将近一个月，袁隆平儿子、国家杂交水稻工...,0,0,9,4
522,杭州检察,06月03日08:12,来自微博视频号,【#袁隆平事迹入选语文教材#】他怀着“禾下乘凉梦”，半个世纪致力于水稻研究。在统编高中语文...,0,3,27,17
523,洮北宣传,06月21日17:05,来自360安全浏览器,袁隆平儿子接棒O袁隆平儿子接棒​,0,0,0,0


In [55]:
result

,发布者ID,发布时间,发布来源,发布内容,收藏数,转发数,评论数,点赞数
0,内蒙古团委,05月23日13:53,来自想见你的Android,【#袁隆平遗体送别仪式明天举行#】5月22日，共和国痛失袁隆平院士。记者从袁隆平同志治丧办...,0,1,1,3
1,正安县检察院,05月25日14:40,来自iPhone客户端,【#女老师哭着读学生的信追思袁老#：当风吹过稻田，我就知道您回来了】#市民赴三亚试验田献花...,0,0,1,0
2,沈阳网警巡查执法,05月24日14:32,来自微博视频号,【#聋哑男孩跑步作画致敬袁隆平#】来自内蒙古自治区赤峰市一名聋哑男孩吴广明，在得知袁隆平院...,0,0,6,5
3,sxrtv-新闻中心,05月22日14:38,来自微博视频号,【痛悼！#送别国之脊梁袁隆平#】今天，#共和国痛失袁隆平院士#。“我最大的愿望是饭碗要牢牢...,0,0,0,0
4,烟台高速交警,05月23日07:16,来自nova7你在焦点在,#送别国之脊梁袁隆平#国士无双……​,0,0,0,1
...,...,...,...,...,...,...,...,...
520,如皋教育发布,05月30日08:16,来自小米6拍人更美,【他们，才是#中国人最该追的星#！致敬！】“憋着一肚子气”搞导弹研发的钱学森、“要求自己鞠...,0,0,0,0
521,江岸检察,06月22日09:46,来自微博视频号,【#袁隆平儿子接棒继续袁梦计划#】距离袁隆平院士逝世将近一个月，袁隆平儿子、国家杂交水稻工...,0,0,9,4
522,杭州检察,06月03日08:12,来自微博视频号,【#袁隆平事迹入选语文教材#】他怀着“禾下乘凉梦”，半个世纪致力于水稻研究。在统编高中语文...,0,3,27,17
523,洮北宣传,06月21日17:05,来自360安全浏览器,袁隆平儿子接棒O袁隆平儿子接棒​,0,0,0,0


In [56]:
result.to_csv('weibo_sample_5255.csv',encoding='utf_8_sig',index=False)

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv('new_weibo_sample_525_4.csv')

In [8]:
data.isnull().sum()

发布者ID                 0
发布时间                  0
发布来源                579
发布内容                  0
收藏数                   0
转发数                   0
评论数                   0
点赞数                   0
verified_reason     590
description        2291
dtype: int64

In [ ]:
data[]